# 🧪 Laboratorio Avanzado: Técnicas de Optimización en Azure Databricks

## 🎯 Objetivo
Aplicar un conjunto de técnicas expertas de rendimiento en Spark y Delta Lake sobre un flujo de datos de ventas, usando un cluster sencillo con configuraciones avanzadas.

## 🧠 Técnicas cubiertas
- Z-Ordering (`ZORDER BY`)
- Optimize con bin packing
- Bloom filters para joins
- Dynamic Partition Pruning
- AQE + Broadcast Joins
- Column pruning
- Cost-based optimization


In [0]:
# Simulación de datos de ventas
from pyspark.sql.functions import expr, rand, col
df_sales = spark.range(0, 10_000_000).withColumn("store_id", (col("id") % 100)) \
    .withColumn("product_id", (col("id") % 1000)) \
    .withColumn("amount", rand() * 100) \
    .withColumn("region", expr("CASE WHEN store_id % 2 = 0 THEN 'North' ELSE 'South' END"))

df_sales.write.format("delta").mode("overwrite").partitionBy("region").save("/mnt/optimizacion/sales")

## 🔁 OPTIMIZE y ZORDER para filtros altamente selectivos

In [0]:
spark.sql("OPTIMIZE delta.`/mnt/optimizacion/sales` ZORDER BY (store_id, product_id)")

## 🧪 Dynamic Partition Pruning + Column Pruning
Habilita DPP y accede solo a columnas necesarias.

In [0]:
spark.conf.set("spark.sql.optimizer.dynamicPartitionPruning", True)
df_filtered = spark.read.format("delta").load("/mnt/optimizacion/sales") \
    .filter("region = 'North' and store_id = 42") \
    .select("product_id", "store_id", "amount")
df_filtered.show(5)

## 📊 AQE + Join Broadcast Auto Tuning
Combina AQE y broadcast inteligente con un join de productos.

In [0]:
spark.conf.set("spark.sql.adaptive.enabled", True)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10 * 1024 * 1024)  # 10MB

df_products = spark.range(0, 1000).withColumnRenamed("id", "product_id")

df_joined = df_filtered.join(df_products, "product_id", "inner")
df_joined.explain(True)

## ✅ Validación
- Revisa Spark UI: ¿se usó ZORDER?
- ¿Se aplicó Dynamic Partition Pruning?
- ¿Se ejecutó el join con broadcast?
- ¿Se redujo el número de archivos tras `OPTIMIZE`?

Este laboratorio pone en práctica técnicas reales para producción en escenarios de gran volumen sin necesidad de escalar recursos.